## Pydantic Fields
https://docs.pydantic.dev/latest/concepts/fields/

In [ ]:
from pydantic import BaseModel, Field

class Model(BaseModel):
    name: str = Field(frozen = True)

We can use ellipsis ```...``` to emphasize the value is required.

```
class Model(BaseModel):
    name: str = Field(..., frozen = True)
```

Pydantic uses ```Annotated``` typing construct to attach metadata to annotation: (although I don't get this Annotated construct yet)

In [1]:
from typing import Annotated

from pydantic import BaseModel, Field, WithJsonSchema


class Model(BaseModel):
    name: Annotated[str, Field(strict=True), WithJsonSchema({'extra': 'data'})]

#### Default Value
Default values can be provided with the ```default``` argument.

In [2]:
from pydantic import BaseModel, Field


class User(BaseModel):
    # Both fields aren't required:
    name: str = 'John Doe'
    age: int = Field(default=20)

#### Default factory

In addition to ```default``` for default values, we can also use ```default_factory```, which can also take a single argument of already validated data, and gets passed in as a dict.

In [ ]:
from pydantic import BaseModel, EmailStr, Field


class User(BaseModel):
    email: EmailStr
    # Use a default factory to set the username based on the email if not provided:
    username: str = Field(default_factory=lambda data: data['email'])


user = User(email='user@example.com')
print(user.username)
#> user@example.com

user@example.com


#### Enable validation default values

By default, Pydantic won't validate default values, but we can enable this behavior by ```validate_default``` argument.

In [5]:
from pydantic import BaseModel, Field, ValidationError


class User(BaseModel):
    age: int = Field(default='twelve', validate_default=True)


try:
    user = User()
except ValidationError as e:
    print(e)
    """
    1 validation error for User
    age
      Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='twelve', input_type=str]
    """

1 validation error for User
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='twelve', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/int_parsing


#### Numeric Constraints

There are some keyword arguments that can be used to constrain numeric values:

* gt - greater than
* lt - less than
* ge - greater than or equal to
* le - less than or equal to
* multiple_of - a multiple of the given number
* allow_inf_nan - allow 'inf', '-inf', 'nan' values

In [6]:
from pydantic import BaseModel, Field


class Foo(BaseModel):
    positive: int = Field(gt=0)
    non_negative: int = Field(ge=0)
    negative: int = Field(lt=0)
    non_positive: int = Field(le=0)
    even: int = Field(multiple_of=2)
    love_for_pydantic: float = Field(allow_inf_nan=True)


foo = Foo(
    positive=1,
    non_negative=0,
    negative=-1,
    non_positive=0,
    even=2,
    love_for_pydantic=float('inf'),
)
print(foo)
"""
positive=1 non_negative=0 negative=-1 non_positive=0 even=2 love_for_pydantic=inf
"""

positive=1 non_negative=0 negative=-1 non_positive=0 even=2 love_for_pydantic=inf


'\npositive=1 non_negative=0 negative=-1 non_positive=0 even=2 love_for_pydantic=inf\n'

#### String Constraints

* min_length: Minimum length of the string.
* max_length: Maximum length of the string.
* pattern: A regular expression that the string must match.

In [7]:
from pydantic import BaseModel, Field


class Foo(BaseModel):
    short: str = Field(min_length=3)
    long: str = Field(max_length=10)
    regex: str = Field(pattern=r'^\d*$')  


foo = Foo(short='foo', long='foobarbaz', regex='123')
print(foo)
#> short='foo' long='foobarbaz' regex='123'

short='foo' long='foobarbaz' regex='123'


### Decimal Constraint

* ```max_digits```: the max number of digits, doesn't count leading or trailing zeros.
* ```decimal_places```: max number of decimal places allowed. doesn't count trailing zeros.

In [1]:
from decimal import Decimal

from pydantic import BaseModel, Field


class Foo(BaseModel):
    precise: Decimal = Field(max_digits=5, decimal_places=2)


foo = Foo(precise=Decimal('123.45'))
print(foo)
#> precise=Decimal('123.45')

precise=Decimal('123.45')
